In [ ]:
#import pandas to play with the dataframe
import pandas as pd
import numpy as np


In [ ]:
#laod the csv files 
df1 = pd.read_csv('dataset1.csv', header= 0)
df2 = pd.read_csv('dataset2.csv', header= 0)

#printing it's shape, top 5 rows, and info
print(df1.shape, df2.shape)
print(df1.head(), df2.head())
print(df1.info(), df2.info())

#check the missing value
print("Missing values per column:")
print(df1.isna().sum())
print(df2.isna().sum())

#checks the duplicate values
print("Duplicate rows in df1:", df1.duplicated().sum())
print("Duplicate rows in df2:", df2.duplicated().sum())

#summary of the data
print("Summary statistics for df1:")
print(df1.describe())

print("Summary statistics for df2:")
print(df2.describe())




(907, 12) (2123, 7)
         start_time  bat_landing_to_food habit  rat_period_start  \
0  30/12/2017 18:37            16.000000   rat  30/12/2017 18:35   
1  30/12/2017 19:51             0.074016  fast  30/12/2017 19:50   
2  30/12/2017 19:51             4.000000  fast  30/12/2017 19:50   
3  30/12/2017 19:52            10.000000   rat  30/12/2017 19:50   
4  30/12/2017 19:54            15.000000   rat  30/12/2017 19:50   

     rat_period_end  seconds_after_rat_arrival  risk  reward  month  \
0  30/12/2017 18:38                        108     1       0      0   
1  30/12/2017 19:55                         17     0       1      0   
2  30/12/2017 19:55                         41     0       1      0   
3  30/12/2017 19:55                        111     1       0      0   
4  30/12/2017 19:55                        194     1       0      0   

        sunset_time  hours_after_sunset  season  
0  30/12/2017 16:45            1.870833       0  
1  30/12/2017 16:45            3.100833     

In [16]:

#Parse time & sort
df1["event_time"] = pd.to_datetime(df1["start_time"], dayfirst=True, errors="coerce")
df2["window_time"] = pd.to_datetime(df2["time"], dayfirst=True, errors="coerce")
df1 = df1.sort_values("event_time").reset_index(drop=True)
df2 = df2.sort_values("window_time").reset_index(drop=True)

#Time-aware join (as-of within 30 min backward)
merged = pd.merge_asof(
    df1, df2,
    left_on="event_time", right_on="window_time",
    direction="backward", tolerance=pd.Timedelta("30min")
)

#Feature engineering
rat_minutes_median = merged["rat_minutes"].median(skipna=True)
merged["rat_pressure"] = np.where(merged["rat_minutes"] >= rat_minutes_median, "High", "Low")
merged["recent_rat"] = (merged["seconds_after_rat_arrival"] <= 60).astype(int)

#Handle missing values
#Primary frame: listwise deletion
listwise = merged.dropna(subset=["bat_landing_to_food", "risk", "rat_pressure"]).copy()

#Secondary frame: mean substitution
mean_sub = merged.copy()
for col in ["bat_landing_to_food", "risk"]:
    if col in mean_sub.columns:
        mean_val = mean_sub[col].mean(skipna=True)
        mean_sub[col] = mean_sub[col].fillna(mean_val)


print("Median rat_minutes (for High/Low split):", rat_minutes_median)
print("Shapes -> merged:", merged.shape, "listwise:", listwise.shape, "mean_sub:", mean_sub.shape)

# Quick preview
print(listwise[["event_time","window_time","bat_landing_to_food","risk","rat_minutes","rat_pressure","recent_rat"]].head())


Median rat_minutes (for High/Low split): 7.783333333
Shapes -> merged: (907, 23) listwise: (907, 23) mean_sub: (907, 23)
           event_time         window_time  bat_landing_to_food  risk  \
0 2017-12-26 20:57:00 2017-12-26 20:43:00                  1.0     0   
1 2017-12-26 20:57:00 2017-12-26 20:43:00                  5.0     0   
2 2017-12-26 21:24:00 2017-12-26 21:13:00                  3.0     0   
3 2017-12-26 21:24:00 2017-12-26 21:13:00                 15.0     1   
4 2017-12-26 21:24:00 2017-12-26 21:13:00                  6.0     0   

   rat_minutes rat_pressure  recent_rat  
0     1.283333          Low           0  
1     1.283333          Low           0  
2     4.866667          Low           0  
3     4.866667          Low           0  
4     4.866667          Low           0  
